In [1]:
from chain_logging import langsmith
from dotenv import load_dotenv

load_dotenv()

langsmith("langchain_RAG")


LangSmith 추적을 시작합니다.
[프로젝트명]
langchain_RAG


### RAG 템플릿 실험

In [1]:
import os
from langchain import hub
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from vectorstore_manager import (
    load_vectorstore,
    create_vectorstore,
)
from config import PDF_PATH, VECTORSTORE_PATH, MODEL_PATH
from utils import get_embeddings, format_docs_with_print


# Load or create vectorstore
if os.path.exists(VECTORSTORE_PATH):
    print("Loading existing vectorstore...")
    embeddings = get_embeddings()
    loaded_vectorstore = load_vectorstore(VECTORSTORE_PATH, embeddings)
else:
    print("Vectorstore not found. Creating a new one...")
    embeddings = get_embeddings()
    loaded_vectorstore = create_vectorstore(PDF_PATH, VECTORSTORE_PATH, embeddings)


# 단계 5: 리트리버 생성
k = 2
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": k})

# 단계 6: 프롬프트 생성
prompt = hub.pull("rlm/rag-prompt")

# 단계7: 언어모델 생성
# Ollama 모델을 불러옵니다.
llm = ChatOllama(model=MODEL_PATH, temperature=0)


# 단계 8: 체인 생성
rag_chain = (
    {"context": retriever | format_docs_with_print, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Vectorstore not found. Creating a new one...


c:\Users\SSAFY\miniconda3\envs\py310\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\SSAFY\miniconda3\envs\py310\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [3]:
from chain_output_stream import stream_response

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
# question = "구글 딥마인드 정책에 대해서 알려주세요"
# question = "미래의 AI 소프트웨어 매출 전망은 어떻게 되나요?"
# question = "YouTube 가 2024년에 의무화 한 것은 무엇인가요?"
# question = "연차는 어떻게 사용할 수 있나요?"
question = "유류비 청구는 어떻게 할 수 있나요? 제가 3박 4일로 부산에 갓다오는데 비용은 대충 어떻게 계산하죠?"

response = rag_chain.stream(question)

# 결과 출력
print(f"[HUMAN]\n{question}\n")
print("===" * 20)
# print(rag_chain["retrieved_docs"])
answer = stream_response(response, return_output=True)

[HUMAN]
유류비 청구는 어떻게 할 수 있나요? 제가 3박 4일로 부산에 갓다오는데 비용은 대충 어떻게 계산하죠?


=== Retrieved Documents ===

Document 1:
..\kisa_pdf\3_06_출장규칙(240214).pdf |  page: 14
그외지역:실비 70,000이내
 
   ※ <삭제2017.12.29>
   ※ <삭제2017.12.29.>
   ※ 자가용승용차를이용하여출장가는경우의교통비는여행구간별철도운임또는버스운임
(통상이용되는대중교통요금 )으로한다. 이때출장자는자가용승용차를이용하여출장을
이행한사실을확인할수있는증거서류인고속도로통행영수증 (왕복), 출장일에출장지소재 
주유소에서결제한신용카드매출전표 , 출장지주차영수증중1개이상의증거서류를갖추어
제출해야한다.  다만, 부득이한사유로자가용승용차를이용한경우에는연료비및통행료를  
지급할수있고이때의부득이한사유및연료비산정기준은공무원여비규정및인사혁신처  
예규를적용하며그내용은아래와같다.<개정2017.12.29., 2021.12.28., 2022.5.27., 2023.3.13.>
      업무형편상
부득이한
사유o산간오지,도서벽지등대중교통수단이 없어자가용을이용할수밖에
없는경우
o출장경로가매우복잡･
다양하여대중교통을사실상이용할수없는경우
o공무목적상부득이한심야시간대이동또는긴급한사유가있는경우
--------------------------------------------------

Document 2:
..\kisa_pdf\3_06_출장규칙(240214).pdf |  page: 28
국내 출장비 정산
출장 변동사항 출장비 정산
소 속 성 명 항 목 신청액 집행액 변동액 비 고
교통비
일  비
식  비
숙박비
계
교통비
일  비
식  비
숙박비
계
교통비
일  비
식  비
숙박비
계
교통비
일  비
식  비
숙박비
계
합 계
--------------------------------------------------


Anser: 

유류비 청구는 자가용승용차를 이용하여 출장가는 경우의 교

### 대화 내용을 기억하는 기능을 추가

In [1]:
import os
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from vectorstore_manager import (
    load_vectorstore,
    create_vectorstore,
    create_conversaion_memory,
)
from config import PDF_PATH, VECTORSTORE_PATH, MODEL_PATH, LOCAL_MODEL_PATH, MODEL_NAME
from utils import get_embeddings, format_docs_with_print, create_prompt
from load_model import load_model, create_model

# Load or create vectorstore
if os.path.exists(VECTORSTORE_PATH):
    print("Loading existing vectorstore...")
    embeddings = get_embeddings()
    loaded_vectorstore = load_vectorstore(VECTORSTORE_PATH, embeddings)
else:
    print("Vectorstore not found. Creating a new one...")
    embeddings = get_embeddings()
    loaded_vectorstore = create_vectorstore(PDF_PATH, VECTORSTORE_PATH, embeddings)

# 대화를 기억할 메모리 객체 생성
memory = create_conversaion_memory("chat_history", "answer")

# 단계 5: 리트리버 생성
k = 2
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": k})

# 단계 6: 프롬프트 생성
prompt = create_prompt()

# 단계7: 언어모델 생성
# Ollama 모델을 불러옵니다.
# llm = ChatOllama(model=MODEL_PATH, temperature=0)

if os.path.exists(LOCAL_MODEL_PATH):
    print("Loading existing Local Model...")
    llm = load_model(LOCAL_MODEL_PATH)
else:
    print("Local Model not found. Creating a new one...")
    create_model(MODEL_NAME,LOCAL_MODEL_PATH)
    llm = load_model(LOCAL_MODEL_PATH)


# 단계 8: 체인 생성
# 단계 8: 체인 생성
rag_chain = (
    {
        "context": retriever | format_docs_with_print,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
    }
    | prompt
    | llm
    | StrOutputParser()
)

Loading existing vectorstore...


c:\Users\SSAFY\miniconda3\envs\py39_cuda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\SSAFY\ssafy\자율프로젝트\S11P31S102\AI\chain\vectorstore_manager.py:42: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


Loading existing Local Model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [2]:
from chain_output_stream import stream_response

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "연차는 어떻게 사용할 수 있나요?"
# question = "유류비 청구는 어떻게 할 수 있나요? 제가 3박 4일로 부산에 갓다오는데 비용은 대충 어떻게 계산하죠?"
# question = "아까 물어본 질문을 다시 알려주세요"

response = rag_chain.stream(question)

# 결과 출력
print(f"[HUMAN]\n{question}\n")
print("===" * 20)
answer = stream_response(response, return_output=True)

# 메모리에 대화 내용을 저장
memory.save_context({"question": question}, {"answer": answer})



[HUMAN]
연차는 어떻게 사용할 수 있나요?



c:\Users\SSAFY\miniconda3\envs\py39_cuda\lib\site-packages\transformers\generation\utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Exception in thread Thread-8:
Traceback (most recent call last):
  File "c:\Users\SSAFY\miniconda3\envs\py39_cuda\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\SSAFY\miniconda3\envs\py39_cuda\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\SSAFY\miniconda3\envs\py39_cuda\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\SSAFY\miniconda3\env


=== Retrieved Documents ===

Document 1:
..\kisa_pdf\3_07_취업규칙(240214).pdf |  page: 6
1. 1주40시간근무자가매년말일기준으로잔여보상휴가가 200시간(25x8시간)을초과하는경우
(단시간근로자는 1주40시간근무자에비례하여계산한다 .)
  2. 계속근로연수가 1년미만의기간에발생하는최대11일의연차유급휴가인경우
  ② 제1항에따라이월·저축된저축연차유급휴가에 대해서는미사용연차유급휴가수당을 지급하
지아니한다 .<신설2021.12.28.>
  ③ 저축한연차유급휴가는 10일이상의장기휴가로사용하거나분할하여  사용할수있다. 단, 10일
이상의장기휴가는 3개월이전에신청하여야한다.<신설2021.12.28.>
제25조의3(연차대출제 )  직원은해당연도의연차유급휴가일수 및잔여보상휴가를모두사용한때에
는다음연도의연차유급휴가중의일부를다음각호의기준에따라미리사용할수있다.<신설
2021.12.28.> <개정2022. 5. 27.>
  1. 1년미만재직: 최대5일
  2. 1년이상2년미만재직: 최대6일
  3. 2년이상3년미만재직: 최대7일
  4. 3년이상4년미만재직: 최대8일
  5. 4년이상재직: 최대10일
--------------------------------------------------

Document 2:
..\kisa_pdf\3_08_계약직원 관리규칙(201229).pdf |  page: 15
1인에게만지급
․급여규정및급여
지급규칙에따라
지급
시간외및
휴일근무
수당정상근무시간외
초과근무를한자o통상임금×1.5÷209×
시간외및휴일근무시간수․급여규정및급여
지급규칙에따라
지급
연차수당연차휴가를
사용하지
않은자o통상임금×1÷209×8×
미사용연차휴가일수
--------------------------------------------------


Anser: 



Empty: 